# Adaptive Retrieval for Enhanced RAG Systems

### 시나리오 예시

질문: “퇴사 이후 연차 정산은 어떻게 이루어지나요?”

→ 분류: 문맥 기반 질문

→ 전략: Step-back Prompting + Contextual Chunking

→ 검색: "퇴직 시 근로기준법 적용", "연차 정산 조건", "예외 조항" 포함 섹션

→ 응답: 문맥 흐름을 반영하여 정리된 맞춤형 설명 제공

## Setting Up the Environment

In [1]:
import numpy as np
import json
import fitz
import re

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: PDF에서 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 전체 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]               # 해당 페이지 가져오기
        text = page.get_text("text")         # 텍스트 형식으로 내용 추출
        all_text += text                     # 추출된 텍스트 누적

    # 추출된 전체 텍스트 반환
    return all_text

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로 분할하되, 각 청크 간에 overlap만큼 겹치게 합니다.

    Args:
        text (str): 분할할 텍스트
        n (int): 각 청크의 문자 수 (기본값: 1000)
        overlap (int): 청크 간 겹치는 문자 수 (기본값: 200)

    Returns:
        List[str]: 분할된 텍스트 청크 리스트
    """
    chunks = []  # 청크들을 저장할 리스트 초기화

    # n - overlap 만큼 이동하면서 청크 생성
    for i in range(0, len(text), n - overlap):
        # i에서 i + n까지의 텍스트 조각을 청크로 추가
        chunks.append(text[i:i + n])

    return chunks  # 생성된 청크 리스트 반환

## Simple Vector Store Implementation

In [5]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 구현체입니다.
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []     # 임베딩 벡터 리스트
        self.texts = []       # 원본 텍스트 리스트
        self.metadata = []    # 각 텍스트의 메타데이터 리스트

    def add_item(self, text, embedding, metadata=None):
        """
        단일 텍스트 항목을 벡터 저장소에 추가합니다.

        Args:
            text (str): 원본 텍스트
            embedding (List[float]): 텍스트의 임베딩 벡터
            metadata (dict, optional): 추가 메타데이터 (기본값: 빈 딕셔너리)
        """
        self.vectors.append(np.array(embedding))  # 임베딩을 NumPy 배열로 변환하여 저장
        self.texts.append(text)                   # 원본 텍스트 저장
        self.metadata.append(metadata or {})      # 메타데이터 저장 (None일 경우 빈 딕셔너리)

    def similarity_search(self, query_embedding, k=5):
        """
        질의 임베딩과 가장 유사한 텍스트를 검색합니다.

        Args:
            query_embedding (List[float]): 질의 임베딩 벡터
            k (int): 반환할 상위 결과 개수

        Returns:
            List[Dict]: 유사한 항목 리스트 (텍스트, 메타데이터, 유사도 포함)
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환

        # 질의 벡터를 NumPy 배열로 변환
        query_vector = np.array(query_embedding)

        # 각 벡터와의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (
                np.linalg.norm(query_vector) * np.linalg.norm(vector)
            )
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 저장

        # 유사도 기준 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개 항목 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })

        return results

## Creating Embeddings

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 임베딩을 생성합니다.

    Args:
        text (str 또는 List[str]): 임베딩을 생성할 입력 텍스트(또는 텍스트 리스트)
        model (str): 사용할 임베딩 모델 이름

    Returns:
        List[float] 또는 List[List[float]]: 생성된 임베딩 벡터 또는 벡터 리스트
    """
    # 입력이 문자열 하나일 수도 있고, 문자열 리스트일 수도 있으므로 리스트 형태로 통일
    input_text = text if isinstance(text, list) else [text]

    # 지정된 모델을 사용하여 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 입력이 단일 문자열이었을 경우, 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding

    # 여러 문자열일 경우, 모든 임베딩 리스트 반환
    return [item.embedding for item in response.data]

## Document Processing Pipeline

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    적응형 검색을 위한 문서 처리 함수

    Args:
        pdf_path (str): 처리할 PDF 파일 경로
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 겹치는 문자 수

    Returns:
        Tuple[List[str], SimpleVectorStore]: 텍스트 청크 리스트와 벡터 저장소
    """
    # PDF 파일에서 텍스트 추출
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    # 텍스트를 일정 길이로 청크 분할
    print("텍스트를 청크 단위로 분할 중...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"{len(chunks)}개의 텍스트 청크 생성 완료")

    # 각 청크에 대해 임베딩 생성
    print("청크에 대한 임베딩 생성 중...")
    chunk_embeddings = create_embeddings(chunks)

    # 벡터 저장소 초기화
    store = SimpleVectorStore()

    # 각 청크와 임베딩, 메타데이터를 저장소에 추가
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"총 {len(chunks)}개의 청크가 벡터 저장소에 추가됨")

    # 텍스트 청크와 벡터 저장소 반환
    return chunks, store

## Query Classification

In [8]:
def classify_query(query, model="gpt-4o-mini"):
    """
    사용자의 질의를 다음 네 가지 중 하나로 분류합니다: Factual, Analytical, Opinion, Contextual

    Args:
        query (str): 사용자 질의
        model (str): 사용할 LLM 모델

    Returns:
        str: 분류된 질의 유형
    """
    # 시스템 프롬프트: LLM에게 분류 기준과 출력 형식을 안내
    system_prompt = """귀하는 질문을 분류하는 전문가입니다.
    주어진 쿼리를 다음 카테고리 중 정확히 한 가지로 분류하세요:
    - 사실: 구체적이고 검증 가능한 정보를 찾는 쿼리.
    - 분석적: 종합적인 분석이나 설명이 필요한 쿼리.
    - 의견: 주관적인 사안에 대한 질의 또는 다양한 관점을 추구하는 질의.
    - 컨텍스트: 사용자별 컨텍스트에 따라 달라지는 쿼리.

    설명이나 추가 텍스트 없이 카테고리 이름만 반환합니다.
    """

    # 사용자 질의를 포함한 프롬프트 구성
    user_prompt = f"Classify this query: {query}"
    
    # LLM을 호출하여 질의 분류 요청
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 분류 결과 추출 및 정제
    category = response.choices[0].message.content.strip()
    
    # 유효한 분류 카테고리 정의
    valid_categories = ["Factual", "Analytical", "Opinion", "Contextual"]
    
    # 응답이 유효한 카테고리 중 하나에 포함되는지 확인
    for valid in valid_categories:
        if valid in category:
            return valid
    
    # 분류 실패 시 기본값으로 "Factual" 반환
    return "Factual"

## Implementing Specialized Retrieval Strategies
### 1. Factual Strategy - Focus on Precision

In [9]:
def factual_retrieval_strategy(query, vector_store, k=4):
    """
    사실 기반 질의에 적합한 검색 전략 (정확도 중심)

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서 수

    Returns:
        List[Dict]: 검색된 문서 목록
    """
    print(f"Executing Factual retrieval strategy for: '{query}'")
    
    # 질의 정밀도를 높이기 위한 LLM 기반 질의 개선 프롬프트 정의
    system_prompt = """귀하는 검색 쿼리를 개선하는 전문가입니다.
    귀하의 임무는 주어진 사실 쿼리를 재구성하여 정보 검색을 위해 더 정확하고
    정보 검색을 위해 구체화하는 것입니다. 주요 엔터티와 그 관계에 집중하세요.

    설명 없이 개선된 쿼리만 제공하세요.
    """

    user_prompt = f"Enhance this factual query: {query}"
    
    # LLM을 통해 개선된 질의 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 개선된 질의 추출 및 출력
    enhanced_query = response.choices[0].message.content.strip()
    print(f"Enhanced query: {enhanced_query}")
    
    # 개선된 질의에 대해 임베딩 생성
    query_embedding = create_embeddings(enhanced_query)
    
    # 유사도 검색을 통해 후보 문서 검색 (후보 수는 2배로 확장)
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)
    
    # 문서별 관련성 평가 결과 저장용 리스트 초기화
    ranked_results = []
    
    # LLM을 통해 각 문서의 질의에 대한 관련성 점수 산정
    for doc in initial_results:
        relevance_score = score_document_relevance(enhanced_query, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "relevance_score": relevance_score
        })
    
    # 관련성 점수를 기준으로 결과 정렬 (내림차순)
    ranked_results.sort(key=lambda x: x["relevance_score"], reverse=True)
    
    # 상위 k개 문서 반환
    return ranked_results[:k]

### 2. Analytical Strategy - Comprehensive Coverage

In [10]:
def analytical_retrieval_strategy(query, vector_store, k=4):
    """
    분석형(Analytical) 질의에 대한 검색 전략: 주제 전반을 포괄하는 정보 중심

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서 수

    Returns:
        List[Dict]: 검색된 문서 목록
    """
    print(f"Executing Analytical retrieval strategy for: '{query}'")
    
    # LLM이 복잡한 질의를 하위 질문으로 분해하도록 유도하는 시스템 프롬프트
    system_prompt = """귀하는 복잡한 질문을 세분화하는 데 전문가입니다.
    기본 분석 쿼리의 다양한 측면을 탐구하는 하위 질문을 생성하세요.
    이러한 하위 질문은 주제를 폭넓게 다루어야 하고
    포괄적인 정보를 검색하는 데 도움이 되어야 합니다.

    정확히 3개의 하위 질문 목록을 한 줄에 하나씩 반환합니다.
    """

    # 사용자 질의를 포함한 프롬프트 생성
    user_prompt = f"이 분석 쿼리에 대한 하위 질문 생성하기: {query}"
    
    # LLM을 통해 하위 질문(sub-questions) 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )
    
    # 응답에서 하위 질문 추출 및 정리
    sub_queries = response.choices[0].message.content.strip().split('\n')
    sub_queries = [q.strip() for q in sub_queries if q.strip()]
    print(f"Generated sub-queries: {sub_queries}")
    
    # 각 하위 질문에 대해 문서 검색 수행
    all_results = []
    for sub_query in sub_queries:
        sub_query_embedding = create_embeddings(sub_query)
        results = vector_store.similarity_search(sub_query_embedding, k=2)
        all_results.extend(results)
    
    # 중복 문서 제거 및 다양한 출처 확보
    unique_texts = set()
    diverse_results = []
    
    for result in all_results:
        if result["text"] not in unique_texts:
            unique_texts.add(result["text"])
            diverse_results.append(result)
    
    # 필요한 문서 수가 부족할 경우, 원래 질의로 직접 검색하여 보완
    if len(diverse_results) < k:
        main_query_embedding = create_embeddings(query)
        main_results = vector_store.similarity_search(main_query_embedding, k=k)
        
        for result in main_results:
            if result["text"] not in unique_texts and len(diverse_results) < k:
                unique_texts.add(result["text"])
                diverse_results.append(result)
    
    # 상위 k개의 결과만 반환
    return diverse_results[:k]

### 3. Opinion Strategy - Diverse Perspectives

In [11]:
def opinion_retrieval_strategy(query, vector_store, k=4):
    """
    의견형 질의에 대한 검색 전략
    다양한 관점을 중심으로 정보를 수집하여 응답의 균형성과 깊이를 높이는 목적

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서 수

    Returns:
        List[Dict]: 검색된 문서 목록
    """
    print(f"의견형 검색 전략 실행 중: '{query}'")

    # LLM이 다양한 관점을 생성하도록 유도하는 시스템 프롬프트
    system_prompt = """귀하는 한 주제에 대한 다양한 관점을 식별하는 데 전문가입니다.
    의견이나 관점에 대한 주어진 쿼리에 대해 사람들이 이 주제에 대해 가질 수 있는 다양한 관점을 식별하세요.
    정확히 3개의 서로 다른 관점을 한 줄에 하나씩 반환하세요.
    """

    # 사용자 질의를 포함한 프롬프트 구성
    user_prompt = f"Identify different perspectives on: {query}"

    # LLM을 통해 관점 목록 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )

    # 응답에서 관점 목록 추출 및 정제
    viewpoints = response.choices[0].message.content.strip().split('\n')
    viewpoints = [v.strip() for v in viewpoints if v.strip()]
    print(f"도출된 관점 목록: {viewpoints}")

    # 각 관점에 대해 문서 검색 수행
    all_results = []
    for viewpoint in viewpoints:
        # 원래 질의에 관점을 결합하여 쿼리를 강화
        combined_query = f"{query} {viewpoint}"
        viewpoint_embedding = create_embeddings(combined_query)
        results = vector_store.similarity_search(viewpoint_embedding, k=2)

        # 검색 결과에 해당 관점 정보 포함
        for result in results:
            result["viewpoint"] = viewpoint

        all_results.extend(results)

    # 관점별로 하나씩 대표 문서를 선정
    selected_results = []
    for viewpoint in viewpoints:
        viewpoint_docs = [r for r in all_results if r.get("viewpoint") == viewpoint]
        if viewpoint_docs:
            selected_results.append(viewpoint_docs[0])

    # 부족한 문서 수는 유사도 순으로 추가 확보
    remaining_slots = k - len(selected_results)
    if remaining_slots > 0:
        remaining_docs = [r for r in all_results if r not in selected_results]
        remaining_docs.sort(key=lambda x: x["similarity"], reverse=True)
        selected_results.extend(remaining_docs[:remaining_slots])

    # 최종 k개의 문서 반환
    return selected_results[:k]

### 4. Contextual Strategy - User Context Integration

In [12]:
def contextual_retrieval_strategy(query, vector_store, k=4, user_context=None):
    """
    컨텍스트 기반 질의에 대한 검색 전략
    사용자 맥락을 통합하여 검색 관련성을 향상시키는 방식

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서 수
        user_context (str): 명시적 또는 질의로부터 추론된 사용자 맥락 정보

    Returns:
        List[Dict]: 검색된 문서 목록
    """
    print(f"컨텍스트 기반 검색 전략 실행 중: '{query}'")

    # 사용자 맥락이 명시되지 않은 경우, LLM을 통해 질의로부터 추론
    if not user_context:
        system_prompt = """귀하는 질문에 내포된 문맥을 이해하는 데 전문가입니다.
        주어진 쿼리에 대해 어떤 문맥 정보가 관련되거나 암시되는지 추론하세요.
        이 쿼리에 답하는 데 도움이 될 수 있는 배경 정보를 중심으로 작성하세요.
        간결한 문장 하나로 암시된 컨텍스트를 반환하세요."""

        user_prompt = f"이 쿼리에서 암시된 컨텍스트 추론하기: {query}"

        # LLM에게 컨텍스트 추론 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )

        # 추론된 사용자 맥락 추출
        user_context = response.choices[0].message.content.strip()
        print(f"추론된 컨텍스트: {user_context}")

    # 질의에 사용자 맥락을 통합하여 질의를 재구성
    system_prompt = """귀하는 문맥에 맞게 질문을 재구성하는 데 전문가입니다.
    쿼리와 주어진 컨텍스트 정보를 기반으로 보다 구체적이고 명확한 질의로 바꿔주세요.
    설명 없이 재작성된 쿼리만 출력하세요."""

    user_prompt = f"""
    Query: {query}
    Context: {user_context}

    이 컨텍스트를 반영하여 쿼리를 다시 작성하세요."""

    # LLM에게 질의 재작성 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 재작성된 컨텍스트 기반 질의 추출
    contextualized_query = response.choices[0].message.content.strip()
    print(f"컨텍스트 반영 질의: {contextualized_query}")

    # 재작성된 질의에 대한 임베딩 생성 및 문서 검색 수행
    query_embedding = create_embeddings(contextualized_query)
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)

    # 검색된 문서들에 대해 문맥 관련성 평가
    ranked_results = []
    for doc in initial_results:
        context_relevance = score_document_context_relevance(query, user_context, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "context_relevance": context_relevance
        })

    # 맥락 관련성 기준으로 정렬 후 상위 k개 문서 반환
    ranked_results.sort(key=lambda x: x["context_relevance"], reverse=True)
    return ranked_results[:k]

## Helper Functions for Document Scoring

In [13]:
def score_document_relevance(query, document, model="gpt-4o-mini"):
    """
    LLM을 사용하여 문서의 질의에 대한 관련성을 평가합니다.

    Args:
        query (str): 사용자 질의
        document (str): 평가할 문서 텍스트
        model (str): 사용할 LLM 모델

    Returns:
        float: 0~10 사이의 관련성 점수
    """
    # LLM에게 문서 관련성 평가 방법을 안내하는 시스템 프롬프트
    system_prompt = """귀하는 문서 관련성을 평가하는 전문가입니다.
    문서와 쿼리의 관련성을 0에서 10까지의 척도로 평가하세요:
    0 = 전혀 관련 없음
    10 = 쿼리를 완벽하게 해결

    0에서 10 사이의 숫자 점수만 반환하고 그 외에는 반환하지 않습니다.
    """

    # 문서가 너무 길 경우 미리보기로 잘라내기
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document
    
    # 사용자 프롬프트 구성: 질의와 문서 제공 후 관련성 점수 요청
    user_prompt = f"""
        Query: {query}

        Document: {doc_preview}

        Relevance score (0-10):
    """
    
    # LLM 호출을 통해 관련성 점수 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 응답으로부터 점수 텍스트 추출
    score_text = response.choices[0].message.content.strip()
    
    # 정규표현식을 이용하여 숫자 추출
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 점수를 0~10 범위로 제한
    else:
        # 점수 추출 실패 시 기본값 반환
        return 5.0

In [14]:
def score_document_context_relevance(query, context, document, model="gpt-4o-mini"):
    """
    질의와 사용자 맥락을 함께 고려하여 문서의 관련성을 평가합니다.

    Args:
        query (str): 사용자 질의
        context (str): 사용자 맥락 정보
        document (str): 평가할 문서 텍스트
        model (str): 사용할 LLM 모델

    Returns:
        float: 0~10 사이의 관련성 점수
    """
    # LLM에게 질의 + 맥락 기반으로 문서 관련성 평가 방법을 안내하는 시스템 프롬프트
    system_prompt = """귀하는 문맥을 고려하여 문서 관련성을 평가하는 전문가입니다.
    문서가 쿼리를 얼마나 잘 처리하는지에 따라 0에서 10까지의 척도로 평가하세요.
    쿼리를 얼마나 잘 처리하는지에 따라 0에서 10까지 평가합니다:
    0 = 전혀 관련 없음
    10 = 주어진 맥락에서 쿼리를 완벽하게 해결함

    0에서 10 사이의 숫자 점수만 반환하고 다른 점수는 반환하지 않습니다.
"""

    # 문서가 너무 길 경우, 앞부분만 사용하여 평가하도록 자름
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document
    
    # 사용자 프롬프트 구성: 질의, 맥락, 문서를 함께 제공하고 점수 요청
    user_prompt = f"""
    Query: {query}
    Context: {context}

    Document: {doc_preview}

    컨텍스트를 고려한 관련성 점수(0~10):
    """
    
    # LLM을 호출하여 점수 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 응답으로부터 점수 텍스트 추출
    score_text = response.choices[0].message.content.strip()
    
    # 정규표현식을 이용하여 숫자 추출
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 점수를 0~10 범위로 제한
    else:
        # 점수 추출 실패 시 기본값 반환
        return 5.0

## The Core Adaptive Retriever

In [15]:
def adaptive_retrieval(query, vector_store, k=4, user_context=None):
    """
    질의 유형에 따라 적절한 검색 전략을 자동으로 선택하여 실행하는 적응형 검색 함수

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 검색할 문서 수
        user_context (str): 사용자 맥락 정보 (컨텍스트 기반 질의에 사용)

    Returns:
        List[Dict]: 검색된 문서 리스트
    """
    # 질의 유형 분류 (Factual, Analytical, Opinion, Contextual 중 하나)
    query_type = classify_query(query)
    print(f"분류된 질의 유형: {query_type}")

    # 질의 유형에 따라 대응하는 검색 전략 실행
    if query_type == "Factual":
        # 사실 기반 정보 검색
        results = factual_retrieval_strategy(query, vector_store, k)
    elif query_type == "Analytical":
        # 분석적 사고나 복합적 개념이 필요한 경우
        results = analytical_retrieval_strategy(query, vector_store, k)
    elif query_type == "Opinion":
        # 다양한 관점이나 의견을 요구하는 질의
        results = opinion_retrieval_strategy(query, vector_store, k)
    elif query_type == "Contextual":
        # 사용자의 맥락 정보가 중요한 질의
        results = contextual_retrieval_strategy(query, vector_store, k, user_context)
    else:
        # 분류되지 않은 경우, 기본적으로 사실 기반 검색 수행
        results = factual_retrieval_strategy(query, vector_store, k)

    # 최종 검색 결과 반환
    return results

## Response Generation

In [16]:
def generate_response(query, results, query_type, model="gpt-4o-mini"):
    """
    질의 유형과 검색된 문서를 기반으로 적절한 응답을 생성하는 함수

    Args:
        query (str): 사용자 질의
        results (List[Dict]): 검색된 문서 목록
        query_type (str): 질의 유형 (Factual, Analytical, Opinion, Contextual 등)
        model (str): 사용할 LLM 모델

    Returns:
        str: 생성된 응답 텍스트
    """
    # 검색된 문서 내용을 하나의 context로 구성
    context = "\n\n---\n\n".join([r["text"] for r in results])

    # 질의 유형에 따라 시스템 프롬프트 설정
    if query_type == "Factual":
        system_prompt = """귀하는 사실 기반 정보를 제공하는 조력자입니다.
        제공된 문맥을 바탕으로 질문에 정확하고 간결하게 답하세요.
        문맥에 정보가 부족하다면 그 한계를 명확히 밝혀주세요."""

    elif query_type == "Analytical":
        system_prompt = """귀하는 분석적 설명을 제공하는 조력자입니다.
        문맥을 종합적으로 분석하고 주제의 여러 측면을 다루어 주세요.
        정보 간 충돌이 있다면 이를 인정하고 균형 있게 설명하세요."""

    elif query_type == "Opinion":
        system_prompt = """귀하는 다양한 관점을 제시하는 조력자입니다.
        주제에 대한 여러 시각을 공정하게 나열하고 편향되지 않게 설명하세요.
        관점이 제한적일 경우 그 점도 함께 알려주세요."""

    elif query_type == "Contextual":
        system_prompt = """귀하는 문맥에 민감하게 반응하는 조력자입니다.
        질문과 문맥을 모두 고려하여 연관성 높은 응답을 제공합니다.
        문맥이 부족하거나 불완전한 경우 그 한계를 분명히 하세요."""

    else:
        system_prompt = """귀하는 유용한 조력자입니다. 문맥을 바탕으로 질문에 답하고, 문맥이 부족할 경우 명확하게 밝혀주세요."""

    # 사용자 프롬프트 구성
    user_prompt = f"""
    Context:
    {context}

    Question: {query}

    위 문맥을 바탕으로 질문에 적절한 응답을 작성하세요.
    """

    # LLM 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    # 최종 응답 반환
    return response.choices[0].message.content

## Complete RAG Pipeline with Adaptive Retrieval

In [17]:
def rag_with_adaptive_retrieval(pdf_path, query, k=4, user_context=None):
    """
    적응형 검색을 활용한 RAG(Retrieval-Augmented Generation) 파이프라인 전체 실행 함수

    Args:
        pdf_path (str): 처리할 PDF 문서 경로
        query (str): 사용자 질의
        k (int): 검색할 문서 수
        user_context (str): (선택 사항) 사용자 맥락 정보

    Returns:
        Dict: 질의, 질의 유형, 검색된 문서, 응답을 포함한 결과 딕셔너리
    """
    print("\n***RAG WITH ADAPTIVE RETRIEVAL***")
    print(f"Query: {query}")
    
    # 1단계: PDF 문서에서 텍스트 추출, 청크 분할, 임베딩 생성
    chunks, vector_store = process_document(pdf_path)
    
    # 2단계: 질의 유형 분류 (Factual, Analytical, Opinion, Contextual)
    query_type = classify_query(query)
    print(f"Query classified as: {query_type}")
    
    # 3단계: 분류된 유형에 따라 적절한 검색 전략 실행
    retrieved_docs = adaptive_retrieval(query, vector_store, k, user_context)
    
    # 4단계: 질의, 검색 문서, 질의 유형을 기반으로 응답 생성
    response = generate_response(query, retrieved_docs, query_type)
    
    # 5단계: 결과 딕셔너리 구성
    result = {
        "query": query,
        "query_type": query_type,
        "retrieved_documents": retrieved_docs,
        "response": response
    }
    
    print("\n***RESPONSE***")
    print(response)
    
    return result

## Evaluation Framework

In [18]:
def evaluate_adaptive_vs_standard(pdf_path, test_queries, reference_answers=None):
    """
    적응형 검색과 표준 검색을 테스트 질의에 대해 비교 평가하는 함수

    이 함수는 다음의 과정을 수행한다:
    - 문서를 전처리하여 텍스트 청크 및 벡터 저장소 생성
    - 각 질의에 대해 표준 검색과 적응형 검색을 모두 수행
    - 참조 정답이 제공된 경우, 생성된 응답과 비교하여 품질을 평가

    Args:
        pdf_path (str): 지식 기반으로 사용할 PDF 문서 경로
        test_queries (List[str]): 테스트용 질의 리스트
        reference_answers (List[str], optional): 응답 평가를 위한 참조 정답 리스트

    Returns:
        Dict: 각 질의에 대한 검색 및 응답 결과, 평가 점수를 포함한 딕셔너리
    """
    print("적응형 검색 vs. 표준 검색 성능 평가 시작")

    # 문서 처리: 텍스트 추출, 청크 분할, 벡터 저장소 구축
    chunks, vector_store = process_document(pdf_path)

    # 결과 저장용 리스트 초기화
    results = []

    # 각 테스트 질의에 대해 검색 및 응답 생성 수행
    for i, query in enumerate(test_queries):
        print(f"\n질의 {i+1}: {query}")

        # 표준 검색 수행
        print("\n표준 검색 실행 중")
        query_embedding = create_embeddings(query)
        standard_docs = vector_store.similarity_search(query_embedding, k=4)
        standard_response = generate_response(query, standard_docs, "General")

        # 적응형 검색 수행
        print("\n적응형 검색 실행 중")
        query_type = classify_query(query)
        adaptive_docs = adaptive_retrieval(query, vector_store, k=4)
        adaptive_response = generate_response(query, adaptive_docs, query_type)

        # 결과 저장
        result = {
            "query": query,
            "query_type": query_type,
            "standard_retrieval": {
                "documents": standard_docs,
                "response": standard_response
            },
            "adaptive_retrieval": {
                "documents": adaptive_docs,
                "response": adaptive_response
            }
        }

        # 참조 정답이 존재할 경우 함께 저장
        if reference_answers and i < len(reference_answers):
            result["reference_answer"] = reference_answers[i]

        results.append(result)

        # 응답 미리보기 출력
        print("\n응답 비교")
        print(f"표준 검색 응답: {standard_response[:200]}...")
        print(f"적응형 검색 응답: {adaptive_response[:200]}...")

    # 참조 정답이 있는 경우 응답 품질 비교 평가 수행
    if reference_answers:
        comparison = compare_responses(results)
        print("\n응답 비교 평가 결과")
        print(comparison)
    else:
        comparison = "참조 정답이 없어 평가 생략됨"

    # 최종 결과 반환
    return {
        "results": results,
        "comparison": comparison
    }

In [19]:
def compare_responses(results):
    """
    표준 검색과 적응형 검색의 응답을 참조 정답과 비교하여 분석하는 함수

    Args:
        results (List[Dict]): 표준 및 적응형 응답이 포함된 결과 리스트

    Returns:
        str: 응답 비교 분석 결과 텍스트
    """
    # AI 모델이 응답 비교를 수행할 수 있도록 시스템 프롬프트 정의
    comparison_prompt = """귀하는 정보 검색 시스템의 전문 평가자입니다.
    각 쿼리에 대한 표준 검색 응답과 적응형 검색 응답을 비교하세요.
    정확성, 관련성, 포괄성, 참조 답변과의 일치성 등을 기준으로 분석하세요.
    각 방식의 강점과 약점에 대해 한글로 자세히 설명하세요."""

    # 분석 결과를 담을 텍스트 초기화
    comparison_text = "표준 검색 vs 적응형 검색 응답 평가\n\n"

    # 각 질의 결과에 대해 비교 수행
    for i, result in enumerate(results):
        # 참조 정답이 없는 경우는 평가 생략
        if "reference_answer" not in result:
            continue

        # 질의 정보 기록
        comparison_text += f"질의 {i+1}: {result['query']}\n"
        comparison_text += f"(질의 유형: {result['query_type']})\n\n"
        comparison_text += f"[참조 정답]\n{result['reference_answer']}\n\n"

        # 표준 검색 응답 기록
        comparison_text += f"[표준 검색 응답]\n{result['standard_retrieval']['response']}\n\n"

        # 적응형 검색 응답 기록
        comparison_text += f"[적응형 검색 응답]\n{result['adaptive_retrieval']['response']}\n\n"

        # 사용자 프롬프트 구성
        user_prompt = f"""
        Reference Answer: {result['reference_answer']}

        Standard Retrieval Response: {result['standard_retrieval']['response']}

        Adaptive Retrieval Response: {result['adaptive_retrieval']['response']}

        두 응답을 비교 분석해주세요.
        """

        # AI 모델을 사용해 비교 분석 생성
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": comparison_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )

        # 분석 결과 추가
        comparison_text += f"[비교 분석 결과]\n{response.choices[0].message.content}\n\n"

    # 전체 비교 분석 결과 반환
    return comparison_text

## Evaluating the Adaptive Retrieval System (Customized Queries)

In [20]:
# 지식 소스 문서의 경로 설정
# 이 PDF 파일은 RAG 시스템이 활용할 정보를 포함하고 있음
pdf_path = "../../dataset/AI_Understanding.pdf"

# 다양한 질의 유형을 포함하는 테스트 질의 정의
# 적응형 검색(adaptive retrieval)이 질의 의도를 어떻게 처리하는지 시연
test_queries = [
    "설명 가능한 AI(XAI)란 무엇인가요?", # 사실 기반 질의 - 정의/구체 정보 요청
    # "How do AI ethics and governance frameworks address potential societal impacts?", # 분석형 질의 - 포괄적 분석 필요
    # "Is AI development moving too fast for proper regulation?", # 의견형 질의 - 다양한 관점 요청
    # "How might explainable AI help in healthcare decisions?", # 문맥형 질의 - 사용자 상황 고려 필요
]

# 보다 철저한 평가를 위한 참조 정답 정의
# 응답 품질을 객관적으로 평가하는 데 사용 가능
reference_answers = [
    "설명 가능한 AI(XAI)는 의사 결정 방식에 대한 명확한 설명을 제공함으로써 AI 시스템을 투명하고 이해하기 쉽게 만드는 것을 목표로 합니다. 이는 사용자가 AI 기술을 신뢰하고 효과적으로 관리하는 데 도움이 됩니다.",
    # "AI 윤리 및 거버넌스 프레임워크는 AI 시스템이 책임감 있게 개발되고 사용될 수 있도록 가이드라인과 원칙을 수립하여 잠재적인 사회적 영향을 해결합니다. 이러한 프레임워크는 공정성, 책임성, 투명성, 인권 보호에 중점을 두어 위험을 완화하고 유익한 결과를 촉진합니다.",
    # "AI 개발이 너무 빠르게 진행되어 적절한 규제가 필요한지에 대한 의견은 다양합니다. 일부에서는 빠른 발전이 규제 노력보다 빨라 잠재적인 위험과 윤리적 문제를 야기한다고 주장합니다. 다른 사람들은 새로운 도전 과제를 해결하기 위해 규제가 함께 진화하면서 혁신이 현재의 속도로 계속되어야 한다고 생각합니다.",
    # "설명 가능한 AI는 AI 기반 추천에 대한 투명하고 이해하기 쉬운 인사이트를 제공함으로써 의료진의 의사결정에 큰 도움을 줄 수 있습니다. 이러한 투명성은 의료 전문가가 AI 시스템을 신뢰하고, 정보에 입각한 결정을 내리고, AI 제안의 근거를 이해함으로써 환자 치료 결과를 개선하는 데 도움이 됩니다."
]

In [21]:
# 적응형 검색과 표준 검색을 비교하는 평가 실행
# 각 질의를 두 방법으로 처리하고 결과를 비교함
evaluation_results = evaluate_adaptive_vs_standard(
    pdf_path=pdf_path,                  # 지식 추출을 위한 소스 문서
    test_queries=test_queries,          # 평가할 테스트 질의 목록
    reference_answers=reference_answers  # 비교를 위한 참조 정답 (옵션)
)

# 결과는 표준 검색과 적응형 검색의 성능을 질의 유형별로 비교하여
# 적응형 전략이 더 나은 결과를 제공하는 경우를 강조함
print(evaluation_results["comparison"])

적응형 검색 vs. 표준 검색 성능 평가 시작
PDF에서 텍스트 추출 중...
텍스트를 청크 단위로 분할 중...
21개의 텍스트 청크 생성 완료
청크에 대한 임베딩 생성 중...
총 21개의 청크가 벡터 저장소에 추가됨

질의 1: 설명 가능한 AI(XAI)란 무엇인가요?

표준 검색 실행 중

적응형 검색 실행 중
분류된 질의 유형: Factual
Executing Factual retrieval strategy for: '설명 가능한 AI(XAI)란 무엇인가요?'
Enhanced query: 설명 가능한 인공지능(XAI)의 정의와 주요 특징은 무엇인가요?

응답 비교
표준 검색 응답: 설명 가능한 AI(XAI)는 AI 시스템을 더욱 투명하고 이해하기 쉽게 만드는 것을 목표로 하는 기술입니다. XAI는 AI 모델이 의사 결정을 내리는 방식에 대한 인사이트를 제공하여 사용자가 AI의 신뢰성과 공정성을 평가할 수 있도록 돕습니다. 이를 통해 AI 시스템에 대한 신뢰와 책임감을 향상시키고, 의사 결정 과정의 투명성을 높이는 것이 중요합니다. X...
적응형 검색 응답: 설명 가능한 AI(XAI)는 AI 시스템을 더욱 투명하고 이해하기 쉽게 만드는 것을 목표로 하는 기술입니다. XAI는 AI 모델이 의사 결정을 내리는 방식에 대한 인사이트를 제공하여 신뢰와 책임감을 향상시키기 위해 개발되고 있습니다. 이를 통해 사용자는 AI의 결정 과정을 이해하고 평가할 수 있게 됩니다....

응답 비교 평가 결과
표준 검색 vs 적응형 검색 응답 평가

질의 1: 설명 가능한 AI(XAI)란 무엇인가요?
(질의 유형: Factual)

[참조 정답]
설명 가능한 AI(XAI)는 의사 결정 방식에 대한 명확한 설명을 제공함으로써 AI 시스템을 투명하고 이해하기 쉽게 만드는 것을 목표로 합니다. 이는 사용자가 AI 기술을 신뢰하고 효과적으로 관리하는 데 도움이 됩니다.

[표준 검색 응답]
설명 가능한 AI(XAI)는 AI 시스템을 더욱 투명하고 이해하기 쉽게 만드는 것을 목표로